In [1]:
import re
import pandas as pd
import numpy as np
import json
#from bs4 import Beautiful Soup
# from nltk.corpus import Beautiful Soup
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
import konlp
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#헤더추가 (title,content,time)

# train1 =pd.read_csv('test6.csv', engine='python',sep="," ,header=None)
train1 =pd.read_csv('test6.csv', engine='python',sep="," )

In [3]:
train1.head()

사진  조여정 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7  \
0  프로필  국세청         과장         전보          자        NaN        NaN        NaN   
1   조이  영화제         참석        봉준호         감독         진원         작가        NaN   
2   사진  클라라         시선         올킬         블루          롱        드레스        NaN   
3   사진    개         부문      노미네이트        기생충        봉준호         감독         환호   
4   사진  함소원         입장        NaN        NaN        NaN        NaN        NaN   

  Unnamed: 8 Unnamed: 9 Unnamed: 10 Unnamed: 11 Unnamed: 12 Unnamed: 13  
0        NaN        NaN         NaN         NaN         NaN         NaN  
1        NaN        NaN         NaN         NaN         NaN         NaN  
2        NaN        NaN         NaN         NaN         NaN         NaN  
3        NaN        NaN         NaN         NaN         NaN         NaN  
4        NaN        NaN         NaN         NaN         NaN         NaN

In [4]:
train1.shape

(2872, 14)

In [5]:
train1.info

<bound method DataFrame.info of          사진     조여정 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6  \
0       프로필     국세청         과장         전보          자        NaN        NaN   
1        조이     영화제         참석        봉준호         감독         진원         작가   
2        사진     클라라         시선         올킬         블루          롱        드레스   
3        사진       개         부문      노미네이트        기생충        봉준호         감독   
4        사진     함소원         입장        NaN        NaN        NaN        NaN   
5        조이     엄태구         지적         남자        NaN        NaN        NaN   
6        조이      진기          주         매력         미모        NaN        NaN   
7        조이     기생충        이정은         엄마         미소        NaN        NaN   
8        조이      공명        영화제          빛         미소        NaN        NaN   
9        조이     영화제        조여정          칸         반한         미소        NaN   
10       조이     영화제         문지         민트         여신        NaN        NaN   
11       사진      공명         훈남  

In [6]:
train1.dtypes

사진             object
조여정            object
Unnamed: 2     object
Unnamed: 3     object
Unnamed: 4     object
Unnamed: 5     object
Unnamed: 6     object
Unnamed: 7     object
Unnamed: 8     object
Unnamed: 9     object
Unnamed: 10    object
Unnamed: 11    object
Unnamed: 12    object
Unnamed: 13    object
dtype: object

In [7]:
#word2vec 파라미터 설명
# workers =프로세스개수
# size= 워드 벡터 특징값수(벡터의 차원을 정한다.)
# min_count =단어에 대한 최소 빈도수(적은 빈도수의 단어는 학습하지 않는다. )
# window =컨테스트 윈도우 크기
# sample=샘플링 비율 (Ex)1e-3 :0.001이 좋은 성능을 낸다

num_workers=4
num_size=300
min_word_count=10
context=10
downsampling=1e-3

In [12]:
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)


In [13]:
from gensim.models import word2vec
print("Training model...")
model=word2vec.Word2Vec(train1,
                       workers=num_workers,
                       size=num_size,
                       min_count=min_word_count,
                       window=context,
                       sample=downsampling,
                       sg=1) #sg=1(skip-gram을 써라)

2019-11-20 20:21:11,364 : INFO : collecting all words and their counts
2019-11-20 20:21:11,365 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2019-11-20 20:21:11,366 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-20 20:21:11,367 : INFO : collected 23 word types from a corpus of 129 raw words and 14 sentences
2019-11-20 20:21:11,368 : INFO : Loading a fresh vocabulary
2019-11-20 20:21:11,369 : INFO : effective_min_count=10 retains 8 unique words (34% of original 23, drops 15)
2019-11-20 20:21:11,370 : INFO : effective_min_count=10 leaves 108 word corpus (83% of original 129, drops 21)
2019-11-20 20:21:11,371 : INFO : deleting the raw counts dictionary of 23 items
2019-11-20 20:21:11,372 : INFO : sample=0.001 downsamples 8 most-common words
2019-11-20 20:21:11,373 : INFO : downsampling leaves estimated 10 word corpus (9.7% of prior 108)
2019-11-20 20:21:11,375 : INF

Training model...


In [20]:
model

In [10]:
#위에 껄로 학습완료

#모델 하이퍼파라미터를 설정한 내용을 모델 이름에 담는 다면 나중에 참고하기에 좋을 것이다.
#모델을 저장하면 Word2Vec.load()를 통해 모델을 다시 사용할 수 있다. 
model_name="300features_10minwords_10context"
model.save(model_name)

2019-11-20 20:15:46,994 : INFO : saving Word2Vec object under 300features_10minwords_10context, separately None
2019-11-20 20:15:46,997 : INFO : not storing attribute vectors_norm
2019-11-20 20:15:46,998 : INFO : not storing attribute cum_table
2019-11-20 20:15:47,001 : INFO : saved 300features_10minwords_10context


In [11]:
# check embedding result
# '감독'이라는 단어와 가장 비슷한(코사인 유사도가 큰) 100개 단어를 출력하라는 지시
print(model.most_similar(positive=["감독"], topn=100))

C:\Users\glass\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
2019-11-20 20:15:47,013 : INFO : precomputing L2-norms of word weight vectors


KeyError: "word '감독' not in vocabulary"

# 모델 결과 탐색

In [17]:
 # 유사도가 없는 단어 추출
model.wv.doesnt_match('감독은 밥을 먹었어'.split())

2019-11-20 20:26:41,177 : WARNING : vectors for words {'감독은', '먹었어', '밥을'} are not present in the model, ignoring these words


ValueError: cannot select a word from an empty list

In [18]:
 # 유사도가 없는 단어 추출
model.wv.doesnt_match('사진 찍어봐'.split())

2019-11-20 20:26:58,631 : WARNING : vectors for words {'찍어봐', '사진'} are not present in the model, ignoring these words


ValueError: cannot select a word from an empty list

In [19]:
# 가장 유사한 단어를 추출
model.wv.most_similar("사진")

# 아래링크의 결과가 나와야함.
#https://programmers.co.kr/learn/courses/21/lessons/1698

KeyError: "word '사진' not in vocabulary"

In [ ]:
# 학습하기 위해서는 같은 형태의 입력값 만들어야함
# 제목마다 단어의 개수가 다르기 때문에 입력값을 하나의 형태로 만들어야함.
# 모든 단어의 벡터값에 대해 평균을 내서 제목 하나당 하나의 벡터로 만드는 방법 ????????????????????????????????????

# 하나의 제목에 대해 전체 단어의 평균값 계산

In [ ]:

# 하나의 제목에 대해 전체 단어의 평균값 계산

#words:단어의 모음인 하나의 리뷰가 들어간다.????????????????????????????????????????
#model:word2vec의 모델을 넣는 곳이며, 우리가 학습한 word3vec 모델이 들어간다.
#num_features:word2vec으로 임베딩할떼 정했던 벡터의 차원 수를 뜻한다. 

words="사진조여정우아한 플라워원피스 입고"
def get_features(words,model,num_features): #여기서 words모지?
    #출력 벡터 초기화
    feature_vector=np.zeros((num_features),dtype=np.float32)
    
    num_words=0
    #어휘사전준비
    index2word_set=set(model.wv.index2word)
    
    for w in words:
        if w in index2words_set:
            num_word +=1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector=np.add(feature_vector,mode[w])

    #문장의 단어 수 만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector=np.divide(feature_vector,num_words)
    return feature_vector

In [ ]:
print(feature_vector)

# 전체 제목에 대해 각 제목의 평균 벡터구하기

In [ ]:
#title_text: 전체 제목 데이터를 입력하는 인자???????????????????
#model:word2vec 모델 입력하는 인자


def get_dataset(title_text,model,num_features):
    dataset=list()
    
    for s in title_text:
        dataset.append(get_features(s,model,num_features))
        
    titleFeatureVecs=np.stack(dataset)
        
    return titleFeatureVecs

In [ ]:
print(titleFeatureVecs)

In [ ]:
#실제 학습에 사용될 입력값
 train_data_vecs=get_dataset(train1,model,num_features)